In [5]:
import pandas as pd
df = pd.read_json ('/kyoto/data.json')
csv_file = df.to_csv ('comparison_results.csv', index = True)

In [ ]:
csv_file = pd.read_csv('comparison_results.csv')
csv_file = csv_file.transpose()

In [7]:
code from https://stackoverflow.com/questions/31328861/replacing-header-with-top-row
new_header = csv_file.iloc[0] #grab the first row for the header
csv_file = csv_file[1:] #take the data less the header row
csv_file.columns = new_header #set the header row as the df header
csv_file

Unnamed: 0,pic1,pic2,which
-NXV9eNdXgaiF-vFK3rq,5,355,1
-NXV9fx7k1lQQS3DtT0A,803,949,0
-NXV9lH466WD6tZgzWft,558,345,1
-NXV9pNCluPp28wF_lON,1202,1344,0
-NXV9pnvHvYNdrU-rVEY,926,777,0
...,...,...,...
-NZ0XNiN8txIMEVg867K,1487,1051,1
-NZ0XNzVM1Vqm6U7lWEv,245,701,0
-NZ0XOAOcvCrO87opmWi,1473,1484,1
-NZ0XOPtsP3C2GnLuEDn,1008,652,0


In [9]:
csv_file.to_csv('comparisons.csv', index = True)

In [10]:
invalid = [128,237,342,422,1171,1316,1332,1384,1389]

In [11]:
csv_file = csv_file[~csv_file['pic1'].isin(invalid)]
csv_file = csv_file[~csv_file['pic2'].isin(invalid)]

In [13]:
csv_file #23332 rows to 23039 rows
csv_file.to_csv('comparisons_cleaned.csv', index = True)

In [14]:
# !pip install trueskill

  Preparing metadata (setup.py) ... done
  Created wheel for trueskill: filename=trueskill-0.4.5-py3-none-any.whl size=18049 sha256=2ebc48aa237b27954d823eba29069207cf6cf35c8ed46623418ff0cdcd273eb2
  Stored in directory: /Users/ryotatorii/Library/Caches/pip/wheels/b1/20/f2/bb1bca2728bad41aa94d5bf7ca2f8148fd00d2c2e3adfe2ea2
Successfully built trueskill


In [15]:
from trueskill import Rating, quality_1vs1, rate_1vs1

In [18]:
# alice, bob = Rating(), Rating()
# print(alice,bob)
# bob, alice = rate_1vs1(bob, alice)
# print(alice,bob)

trueskill.Rating(mu=25.000, sigma=8.333) trueskill.Rating(mu=25.000, sigma=8.333)
trueskill.Rating(mu=20.604, sigma=7.171) trueskill.Rating(mu=29.396, sigma=7.171)


In [25]:
# dir(alice)
# getattr(alice, 'mu')

20.604167980000835

In [28]:
rl = [Rating() for i in range(1500)]
csv_file = pd.read_csv('comparisons_cleaned.csv')
csv_file = csv_file.reset_index()  # make sure indexes pair with number of rows

In [29]:
# csv_file

,Unnamed: 0,pic1,pic2,which
0,-NXV9eNdXgaiF-vFK3rq,5,355,1
1,-NXV9fx7k1lQQS3DtT0A,803,949,0
2,-NXV9lH466WD6tZgzWft,558,345,1
3,-NXV9pNCluPp28wF_lON,1202,1344,0
4,-NXV9pnvHvYNdrU-rVEY,926,777,0
...,...,...,...,...
23034,-NZ0XNiN8txIMEVg867K,1487,1051,1
23035,-NZ0XNzVM1Vqm6U7lWEv,245,701,0
23036,-NZ0XOAOcvCrO87opmWi,1473,1484,1
23037,-NZ0XOPtsP3C2GnLuEDn,1008,652,0


In [32]:
for index, row in csv_file.iterrows():
    if row['which'] == 0:
        rl[row['pic1']],rl[row['pic2']] = rate_1vs1(rl[row['pic1']],rl[row['pic2']])
    else:
        rl[row['pic2']],rl[row['pic1']] = rate_1vs1(rl[row['pic2']],rl[row['pic1']])
#     print(row['c1'], row['c2'])

In [35]:
rl = [getattr(i, 'mu') for i in rl]

In [47]:
data = pd.read_csv('data.csv')
data['valid'] = 0
data.loc[invalid,'valid'] = 1
data['rating'] = rl

In [49]:
data.to_csv('data_val.csv')

In [50]:
data_sorted = data.sort_values('rating')
data_sorted.to_csv('data_rating_sorted.csv')

In [53]:
# !pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 9.6 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 12.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.2 MB/s eta 0:00:00


In [56]:
from sklearn.preprocessing import MinMaxScaler

pd_data = pd.read_csv('data_val.csv')
scaler = MinMaxScaler()
pd_data["scaled_rating"] = scaler.fit_transform(pd_data[["rating"]])

pd_data.to_csv('data_val_scaled.csv')